In [1]:
from skimage.filters import threshold_local,threshold_otsu,try_all_threshold,threshold_mean,threshold_yen
import numpy as np
import cv2
from skimage.draw import rectangle
from skimage.morphology import skeletonize
from math import sqrt

In [9]:
def OrderPoints(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    n=60
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]+n
    rect[2] = pts[np.argmax(s)]-n

    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]-n
    #x,y
    rect[1][0]-=n
    rect[1][1]+=n
    rect[3] = pts[np.argmax(diff)]
    #x,y
    rect[3][0]+=n
    rect[3][1]-=n
    return rect

def TransformPrespective(image, pts):
    rect = OrderPoints(pts)
    (tl, tr, br, bl) = rect
 
 
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
  [0, 0],
  [maxWidth - 1, 0],
  [maxWidth - 1, maxHeight - 1],
  [0, maxHeight - 1]], dtype = "float32")
 
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
    return warped

In [10]:
def GetMaxContour(img_dir,MORPH=9,CANNY=84,debug=False):
    """
    takes image directory ,
    structural rect dim(optional),canny max thres(optional)
    debug if true outputs images after each process
    returns a boolean specifying if the prespective shoud be adjusted,
    max contour approximation and grayoriginal image
    """
    img = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    img_copy = img.copy()
    hImg, wImg = img_copy.shape
    #smooth image
    cv2.GaussianBlur(img, (3,3), 0, img)
    # Remove small details and writing on paper
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(MORPH,MORPH))
    dilated = cv2.dilate(img, kernel)
    edges = cv2.Canny(dilated, 0, CANNY, apertureSize=3)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    #get edges again so we get inner line surrounding the paper
    #avoid paper outer edges being connected to noise in background
    edges = cv2.dilate(edges, kernel)
    edges = cv2.Canny(edges, 0, CANNY, apertureSize=3)
    # finding contours
    _,contours,heirarchy = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    maxArea=0
    adjustPrespective = False
    approx=[]
    # filter for max contour
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if(w*h >= maxArea):
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.05 * peri, True)
            maxArea=w*h
            if(maxArea/(hImg*wImg) >= 0.1):
                adjustPrespective= True

    if(debug):
        orig = cv2.imread(img_dir)
        cv2.drawContours(orig,[approx], -1, (0, 255, 0), 1)
        cv2.imwrite("orig.png",img)
        cv2.imwrite("dilated.png",dilated)
        cv2.imwrite("edges.png",edges)
        cv2.imwrite('outline.png',orig)
    return adjustPrespective,approx,img_copy

In [11]:
def warpedPrespective(imgToWarp,approxContour,ratio=1,debug=False):
    warped = TransformPrespective(imgToWarp, approxContour.reshape(len(approxContour), 2) * ratio)
    if(debug):
        cv2.imwrite('result.png',warped)
    return warped

In [12]:
def RemoveShadow(img_gray,debug=False):
    #remove noise and writting on paper to leave out onlu the illumination
    dilated_img = cv2.dilate(img_gray, np.ones((7,7), np.uint8)) 
    blurred_img = cv2.medianBlur(dilated_img, 21)
    #remove any dark lighning aka shadow
    diff_img = 255 - cv2.absdiff(img_gray, blurred_img)
    norm_img = diff_img.copy()
    #normalization and word sharpening
    cv2.normalize(diff_img, norm_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    _, thr_img = cv2.threshold(norm_img, 230, 0, cv2.THRESH_TRUNC)
    cv2.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    if(debug):
        cv2.imwrite("shadowOut.png",norm_img)
        cv2.imwrite("thres.png",thr_img)
    return thr_img


In [13]:
def Binarize(img_gray,debug=False):
    thres =threshold_yen(img_gray)
    final = (img_gray >thres)*255
    if(debug):
        cv2.imwrite("final.png",final)
        #fig, ax = try_all_threshold(thr_img, figsize=(20, 30), verbose=False)
    return final


In [25]:
#remove lines function
def FloodFromCorners(im_th,debug=False):
    hImg, wImg = im_th.shape[:2]
    mask = np.zeros((hImg+2, wImg+2), np.uint8)
    im_floodfill = 255 - im_th
    ##################Flood fill corners#####
    cv2.floodFill(im_floodfill, mask, (0,0), 255)
    cv2.floodFill(im_floodfill, mask, (wImg-1,0), 255)
    cv2.floodFill(im_floodfill, mask, (0,hImg-1), 255)
    cv2.floodFill(im_floodfill, mask, (wImg-1,hImg-1), 255)
    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    if(debug):
        cv2.imwrite("Inverted_Floodfilled_Image.png", im_floodfill_inv)
        cv2.imwrite("Floodfilled_Image.png", im_floodfill)
    return im_floodfill_inv


######################cv contours#########
def getClosedShapes(im_filled,debug=False):
    hImg, wImg = im_filled.shape
    _,contours_cv, hierarchy = cv2.findContours(im_filled, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy[0]
    #[Next, Previous, First_Child, Parent]
    im_empty = np.ones((hImg, wImg,3), np.uint8) * 255
    i=0
    filtered_contoures = []
    for cnt in contours_cv:
        if hierarchy[i][3] != -1:
            i+=1
            continue

        #contours that has at least two children
        x,y,w,h = cv2.boundingRect(cnt)
        acceptedArea = h<=150 and w <=300 and h>=7 and w>20
        child1Idx = hierarchy[i][2]
        hasChildren = child1Idx != -1 and  hierarchy[child1Idx][0] != -1
        if(hasChildren and acceptedArea):
            child2Idx = hierarchy[child1Idx][0]
        #this two children are close (letters)
        #check area
        if(hasChildren and acceptedArea):
            #print((w*h)/area)
            x1,y1,w1,h1 = cv2.boundingRect(contours_cv[child1Idx])
            x2,y2,w2,h2 = cv2.boundingRect(contours_cv[child2Idx])
            if(sqrt((x1-x2)**2+((y1-y2)**2)) <= 200):
                cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
                filtered_contoures.append(cnt)
                i+=1
                continue
                
        if(child1Idx != -1 and  hierarchy[child1Idx][0] == -1):
            x1,y1,w1,h1 = cv2.boundingRect(contours_cv[child1Idx])
            if(w1/w >= 0.3):
                cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
                filtered_contoures.append(cnt)
                i+=1
                continue
                
        if(not hasChildren and acceptedArea and w/h >= 1.8 and w/h<=8):
            print(w/h,w,h)
            filtered_contoures.append(cnt)
            cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
            i+=1
            continue
            

        i+=1
    if(debug):
        cv2.imwrite("contoured.png", im_empty[:,:,0])

    return im_empty[:,:,0] , filtered_contoures

In [119]:
# find center of gravity from the moments:
def scale_contours(contours, scale):
    """Shrinks or grows an array of contours by the given factor (float). 
    Returns the resized array of contours"""
    for idx,contour in enumerate(contours):
        moments = cv2.moments(contour)
        midX = int(round(moments["m10"] / moments["m00"]))
        midY = int(round(moments["m01"] / moments["m00"]))
        mid = np.array([midX, midY])
        contours[idx] = contour - mid
        contours[idx] = (contours[idx] * scale).astype(np.int32)
        contours[idx] = contours[idx] + mid
    return contours

def isOverlapped(c,imgContours):
    cXmin,cYmin,cXmax,cYmax = cv2.boundingRect(c)
    cArea = cXmax * cYmax
    cXmax += cXmin
    cYmax += cYmin
    for imgContour in imgContours:
        Xmin,Ymin,Xmax,Ymax = cv2.boundingRect(imgContour)
        Xmax += Xmin
        Ymax += Ymin
        if(cXmin > Xmin and cXmax < Xmax and cYmin >= Ymin and cYmax <= Ymax):
            return True
    return False
def getOpenedContours(img,closed_contours=[],debug = False):
    closed_contours = scale_contours(closed_contours,1.2)
    cv2.drawContours(img,closed_contours,-1, 255,-1 )
    _,contours, hierarchy = cv2.findContours(255 - img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img_parent = np.ones(img.shape, np.uint8) * 255
    parentContours =[]
    for i,cnt in enumerate(contours):
        x,y,w,h = cv2.boundingRect(cnt)
        if not isOverlapped(cnt,contours) and hierarchy[0,i,3]==-1 and h>10 and w>15 and w/h>1.5 and w/h<4:
            parentContours.append(cnt)

    cv2.drawContours(img_parent,parentContours,-1, 0, -1 )
    img_parent = 255 - img_parent 
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    dilated = cv2.dilate(img_parent,kernel,iterations=3)
    edges = cv2.Canny(dilated, 0, 84, apertureSize=3)
    #get child contours and draw hull
    img_inner_contour = np.ones(img.shape, np.uint8) * 255
    _,contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [cnt for i,cnt in enumerate(contours) if isOverlapped(cnt,contours) ]
    cv2.drawContours(img_inner_contour,contours,-1, 0, 1 )
    eroded = cv2.erode(img_inner_contour,kernel,iterations=3)
    _,contours, hierarchy = cv2.findContours(255-eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    opened_contours = np.ones(img.shape, np.uint8) * 255

    cv2.drawContours(opened_contours,contours,-1, 0, 1 )
    if(debug):
        cv2.imwrite("contoured_img.png",img)
        cv2.imwrite("img_parent.png",img_parent)
        cv2.imwrite("dilated.png",dilated)
        cv2.imwrite("edges.png",edges)
        cv2.imwrite("img_inner_contour.png",img_inner_contour)
        cv2.imwrite("eroded.png",eroded)
        cv2.imwrite("opened_contours.png",opened_contours)
    return opened_contours,contours



In [121]:
#integerated preprocessing
img_dir ="input/11.png"
adjustPrespective,approxContour,grayImg = GetMaxContour(img_dir)
warpedImg = grayImg
if(adjustPrespective):
    warpedImg = warpedPrespective(grayImg,approxContour)
shadowFreeImg = RemoveShadow(warpedImg,True)
binarizedImg = Binarize(shadowFreeImg,True)
filledImg = FloodFromCorners(binarizedImg.astype(np.uint8).copy(),True)
contourdImg,filtered_contoures = getClosedShapes(filledImg,True)
image_result = binarizedImg.astype(np.uint8).copy()
contourdImg,opened_contours = getOpenedContours(image_result,filtered_contoures,True)


3.2 32 10


In [ ]:
# hull=[]
# for c in contours:
#     hull.append(cv2.convexHull(c, False))
# cv2.drawContours(im_empty,hull,-1, 0, 1 )
#stacked_img = np.stack((stacked_img,)*3, axis=-1)

# lines = cv2.HoughLinesP(image_result, 1, np.pi/180, 30,0)
# for line in lines:
#     for x1,y1,x2,y2 in line:
#         cv2.line(stacked_img,(x1,y1),(x2,y2),(0,255,0),1)


# lines = cv2.HoughLines(image_result, 1, np.pi/180, 100)
# for line in lines:
#     for r,theta in line:
#         a = np.cos(theta)
#         b=np.sin(theta)
#         x0 = a*r
#         y0 = b*r
#         x1 = int(x0 + 1000*(-b))
#         y1 = int(y0 + 1000*(a))
#         x2 = int(x0 - 1000*(-b))
#         y2 = int(y0 - 1000*(a))

#         cv2.line(stacked_img, (x1,y1), (x2,y2), (0,0,255), 1)

# cv2.imwrite("lines.png",stacked_img)
# print(len(lines))

# for cnt in contours:
#     x,y,w,h = cv2.boundingRect(cnt)
#     rr, cc = rectangle(start = (y,x), end = (y+h,x+w), shape=contourdImg.shape)
#     print(x,y,h,w)
#     if(x==0 and y==0 ):
#         continue
#     n=10
#     x = max(0,x-n)
#     y = max(0,y-n)
#     y2 = min(y+h+n,image_result.shape[0])
#     x2 = min(x+w+n,image_result.shape[1])
#     image_result[y:y2,x:x2] = 255 